## Implementation of Gibbs Sampling to approximate probability queries from a Bayesian Network.

In [2]:
import numpy as np
from calculations_helper import break_up_polytree, handle_dag_gibbs_sampling, join_distributions

def estimate_gibbs(iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Generate an estimate for the probability distribution for a given set of query variables and evidence values

    Args:
        iterations (int): number of samples to take before we go with the estimate
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables with their respective values

    Returns:
        np.array: estimated probability distribution for the different combinations the query variables can take on
    """
    dag_map, query_collections, evidence_collections = break_up_polytree(network, queries, evidence)
    
    if len(dag_map) == 1:
        # only one directed acyclic graph
        return queries, handle_dag_gibbs_sampling(iterations, network, queries, evidence)
    else:
        # each directed acyclic graph will output a probability distribution - we must join them all and keep track of the variables present
        reordered_queries = []
        prob_distributions = []
        for i, dag in dag_map.items():
            these_queries = query_collections[i]
            for v in these_queries:
                reordered_queries.append(v)
            this_evidence = {v:evidence[v] for v in evidence_collections[i]}
            prob_distributions.append(handle_dag_gibbs_sampling(iterations, dag, these_queries, this_evidence))
        return reordered_queries, join_distributions(prob_distributions)

In [3]:
import json

queries = [0]
evidence = {3:True,4:True}

with open('bn_test_1.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

([0], array([0.6901, 0.3099]))


In [4]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

([0, 3], array([0.3434, 0.0686, 0.0999, 0.4881]))


In [5]:
import json

queries = [1]
evidence = {2:False}

with open('bn_test_3.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

([1], array([0.8792, 0.1208]))


In [6]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('small_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

([1, 3], array([5.000e-04, 9.375e-01, 2.400e-03, 5.960e-02]))


In [7]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('big_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

([1, 3], array([0.7386, 0.0847, 0.1451, 0.0316]))
